In [2]:
import altair as alt
from altair.datasets import data

movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={'Release Date':'date'})
)
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = [
    'Action', 'Adventure', 'Black Comedy', 'Comedy',
    'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
    'Romantic Comedy', 'Thriller/Suspense', 'Western'
]

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Big_Budget_Film = "datum['Production Budget'] > 100000000 ? 'Yes' : 'No'",
    Release_Year = "year(datum['Release Date'])",
).transform_filter(
    alt.datum['IMDB Rating'] > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide Gross:Q').scale(domain=(100000,10**9), clamp=True),
    y='IMDB Rating:Q',
    tooltip='Title:N'
)

# A slider filter
year_slider = alt.binding_range(min=1969, max=2018, step=1, name='Release Year')
slider_selection = alt.selection_point(bind=year_slider, fields=['Release_Year'])

filter_year = base.add_params(
    slider_selection
).transform_filter(
    slider_selection
).properties(title='Slider Filtering')

# A dropdown filter
genre_dropdown = alt.binding_select(options=genres, name='Genre')
genre_select = alt.selection_point(fields=['Major Genre'], bind=genre_dropdown)

filter_genres = base.add_params(
    genre_select
).transform_filter(
    genre_select
).properties(title='Dropdown Filtering')

# Color changing marks
rating_radio = alt.binding_radio(options=ratings, name='Rating')
rating_select = alt.selection_point(fields=['MPAA Rating'], bind=rating_radio)

rating_color = (
    alt.when(rating_select)
    .then(alt.Color('MPAA Rating:N').legend(None))
    .otherwise(alt.value('lightgray'))
)

highlight_ratings = base.add_params(
    rating_select
).encode(
    color=rating_color
).properties(title='Radio Button Highlighting')

# Boolean selection for format changes
input_checkbox = alt.binding_checkbox(name='Big Budget Films ')
checkbox_selection = alt.param(bind=input_checkbox)

size_checkbox = (
    alt.when(checkbox_selection)
    .then(alt.Size('Big Budget Film:N').scale(range=[25, 150]))
    .otherwise(alt.value(25))
)

budget_sizing = base.add_params(
    checkbox_selection
).encode(
    size=size_checkbox
).properties(title='Checkbox Formatting')

(filter_year | budget_sizing) & (highlight_ratings | filter_genres)

alt.VConcatChart(...)

In [11]:
import pandas as pd
def get_data():
    return pd.read_csv("./sv_unpiv.csv")

def format_data(stock_data):
    # stock_data = stock_data[stock_data["Code"] == "AFG"]
    stock_data["value"] = stock_data["value"] / 1000000
    return stock_data
stock_data = get_data()
stock_data_formatted = format_data(stock_data)
# stock_data["variable"] = stock_data["variable"].map(lambda x: x.replace("Greenhouse gas emissions from ", ""))


hover = alt.selection_single(
    fields=["Year"],
    nearest=True,
    on="mouseover",
    empty="none",
)
hover = alt.selection_point(name="highlight", on="pointerover", empty=False, nearest=True)
lines = (
    alt.Chart(stock_data_formatted, title="Evolution des émissions de CO2")
    .mark_line()
    .encode(
        x=alt.X("Year", title="Année"),
        y=alt.Y("value", title="Emissions en Gigatonnes"),
        color="variable",
    )
)
points = lines.transform_filter(hover).mark_circle(size=65)


# A dropdown filter
country_dropdown = alt.binding_select(options=["France"], name='Pays')
country_select = alt.selection_point(fields=['Entity'], bind=country_dropdown)
filter_country = points.add_params(
    country_select
)

tooltips = (
    alt.Chart(stock_data_formatted)
    .mark_rule()
    .encode(
        x="Year",
        y="value",
        opacity=alt.condition(hover, alt.value(0.3), alt.value(0)),
        tooltip=[
            alt.Tooltip("Year", title="Year"),
            alt.Tooltip("variable", title="Secteur"),
            alt.Tooltip("value", title="Emissions CO2"),
        ],
    )
    .add_params(hover)
)
data_layer = lines + points + tooltips
data_layer

/tmp/ipykernel_10011/3057520783.py:14: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  hover = alt.selection_single(


MaxRowsError: The number of rows in your dataset (67640) is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.LayerChart(...)

In [ ]:
movies = alt.UrlData(
    data.movies.url,
    format=alt.CsvDataFormat)
)
movies.to_dict()

{'url': 'https://cdn.jsdelivr.net/npm/vega-datasets@v3.2.1/data/movies.json',
 'format': {'parse': {'Release Date': 'date'}}}